In [530]:
import argparse
import io
import json
import os
import sys
from typing import Final, List

import pandas as pd
import pendulum
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame
import urllib3
from urllib3.exceptions import InsecureRequestWarning
from requests_kerberos import OPTIONAL, HTTPKerberosAuth
urllib3.disable_warnings()

import datetime
from datetime import datetime, timedelta




In [531]:
today = datetime.now()

count = 0

whid = ['VIL2', 'VIL3', 'VMI1', 'VFL4', 'VIL1']
days = ['Saturday', 'Friday', 'Thursday', 'Wednesday', 'Tuesday', 'Monday', 'Sunday']
VolumeAM = ['Volume01.csv', 'Volume02.csv', 'Volume03.csv', 'Volume04.csv', 'Volume05.csv', 'Volume06.csv', 'Volume07.csv']
VolumePM = ['Volume01a.csv', 'Volume02a.csv', 'Volume03a.csv', 'Volume04a.csv', 'Volume05a.csv', 'Volume06a.csv', 'Volume07a.csv']

In [532]:
def intro():
    global count
    while count < 1:
        print("""
                Welcome to the Volume Retriever v2.0
                Brought to you by: Gerber Albizures (galbizur@amazon.com)
                New to v2.0, recursion introduced for VIL2, VIL3, SMI1, VFL4, VIL1
                Multithreading reducing overall times.
                Excel color map created for visualization purposes to emulate excel processing
                Visualizations will be saved to produce email.
        """)
    #global count
        count = count + 1
    
intro()


                Welcome to the Volume Retriever v2.0
                Brought to you by: Gerber Albizures (galbizur@amazon.com)
                New to v2.0, recursion introduced for VIL2, VIL3, SMI1, VFL4, VIL1
                Multithreading reducing overall times.
                Excel color map created for visualization purposes to emulate excel processing
                Visualizations will be saved to produce email.
        


In [533]:
def dow(date):
    days=["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
    dayNumber=date.weekday()
    print(dayNumber)
    return dayNumber
    
def getSaturday():
    today = datetime.now()
    date = today.weekday()
    if date == 0:
        d = today - timedelta(days=2)
        return d
    elif date == 1:
        d = today - timedelta(days=3)
        #print(type(d))
        return d
    elif date == 2:
        d = today - timedelta(days=4)
        return d
    elif date == 3:
        d = today - timedelta(days=5)
        return d
    elif date == 4:
        d = today - timedelta(days=6)
        return d
    elif date == 5:
        d = today - timedelta(days=7)
        return d
    else:
        d = today - timedelta(days=1)
        return d
        

lastSaturday = (getSaturday())
lastSunday = getSaturday() - timedelta(days=6)

strLastSaturday = lastSaturday.strftime('%Y-%m-%d')


strLastSunday = lastSunday.strftime('%Y-%m-%d')


In [534]:
def DayShiftVolume():
    if count == 1:     
        for timeG in range(7):
            lastSunday = getSaturday() - timedelta(days=timeG)
            strLastSaturday = lastSaturday.strftime('%Y-%m-%d')
            strLastSunday = lastSunday.strftime('%Y-%m-%d')

            
            weekDay = days[timeG]

            print(str(datetime.now())+": Starting Volume pulls from last", weekDay, "day shift.")
            date1 = strLastSunday
            year1, month1, day1 = map(str, strLastSunday.split('-'))
            time1 = "06:30"
            hour1, minute1 = map(str, time1.split(':'))

            date2 = date1
            year2, month2, day2 = map(str, date2.split('-'))
            time2 = "20:00"
            hour2, minute2 = map(str, time2.split(':'))
            dateTotal = str(date1)+ " "+str(time1)+"-" +str(time2)
            my_dict = {"DS":[], dateTotal:[]}

            for x in whid:
                fclm_url = f'http://fclm-portal.amazon.com/ppa/inspect/node/csv?nodeType=DS&warehouseId={x}&startDateDay=2021%2F04%2F30&startDateWeek=2021%2F06%2F06&startDateMonth=2021%2F04%2F01&maxIntradayDays=1&spanType=Intraday&startDateIntraday={year1}%2F{month1}%2F{day1}&startHourIntraday={hour1}&startMinuteIntraday={minute1}&endDateIntraday={year2}%2F{month2}%2F{day2}&endHourIntraday={hour2}&endMinuteIntraday={minute2}'
                req = requests.Session()
                resp = req.get(fclm_url, verify=False, allow_redirects=True, auth=HTTPKerberosAuth(mutual_authentication=OPTIONAL))
                data = io.StringIO(resp.text)
                df = pd.read_csv(data)
            #print(df.info)
                lfilter = df['Labor Process Name']=="Container Building" 
                df.where(lfilter, inplace=True)#1
                filter = df['Main Processes'] == 'In-Station Core'
                df.where(filter, inplace=True)
                DS_sum = df["Actual Units"].sum()

                if x == whid[-1] and today.isocalendar()[1] < 32:
                    if DS_sum == 0:
                        print(str(datetime.now())+":", weekDay, "day shift pull request for", x, "dropped, retry in place.")
                        req = requests.Session()
                        resp = req.get(fclm_url, verify=False, allow_redirects=True, auth=HTTPKerberosAuth(mutual_authentication=OPTIONAL))
                        data = io.StringIO(resp.text)
                        df = pd.read_csv(data)
                            #print(df.info)
                        lfilter = df['Labor Process Name']=="Container Building" 
                        df.where(lfilter, inplace=True)#2
                        filter = df['Main Processes'] == 'In-Station Core'
                        df.where(filter, inplace=True)
                        DS_sum = df["Actual Units"].sum()
                        #print(str(datetime.now())+":", weekDay, "day shift pull for", x, "completed.")

                else:
                    while DS_sum == 0:
                        print(str(datetime.now())+":", weekDay, "day shift pull request for", x, "dropped, retry in place.")
                        req = requests.Session()
                        resp = req.get(fclm_url, verify=False, allow_redirects=True, auth=HTTPKerberosAuth(mutual_authentication=OPTIONAL))
                        data = io.StringIO(resp.text)
                        df = pd.read_csv(data)
                        #print(df.info)
                        lfilter = df['Labor Process Name']=="Container Building" 
                        df.where(lfilter, inplace=True)#2
                        filter = df['Main Processes'] == 'In-Station Core'
                        df.where(filter, inplace=True)
                        DS_sum = df["Actual Units"].sum()
                print(str(datetime.now())+":", weekDay, "day shift pull for", x, "completed.")


                my_dict["DS"].append(x)
                my_dict[dateTotal].append(DS_sum)
                #timeG = timeG + 1

            Volume1 = pd.DataFrame.from_dict(my_dict, orient='index', columns = ['VIL2', 'VIL3', 'VMI1', 'VFL4', 'VIL1'])
            Volume1.drop(labels = 'DS', axis = 0, inplace=True)

            Volume1 = Volume1.reset_index()
            Volume1.to_csv(VolumeAM[timeG])
            if timeG==6:
                Vol1 = pd.read_csv(VolumeAM[0])
                Vol2 = pd.read_csv(VolumeAM[1])
                Vol3 = pd.read_csv(VolumeAM[2])
                Vol4 = pd.read_csv(VolumeAM[3])
                Vol5 = pd.read_csv(VolumeAM[4])
                Vol6 = pd.read_csv(VolumeAM[5])
                Vol7 = pd.read_csv(VolumeAM[6])
                df_dayShift = pd.concat([Vol1, Vol2, Vol3, Vol4, Vol5, Vol6, Vol7], ignore_index=True)
                df_dayShift = df_dayShift.drop(['Unnamed: 0'], axis=1)
                df_dayShift.to_csv("DayShift.csv")
                
                for x in VolumeAM:
                    os.remove(x)
                    print(str(datetime.now())+":", x, "has been removed.")
                    
                

                return df_dayShift
v1 = DayShiftVolume()
v1
        

2021-07-30 00:40:39.541197: Starting Volume pulls from last Saturday day shift.
2021-07-30 00:41:06.145124: Saturday day shift pull for VIL2 completed.
2021-07-30 00:41:33.135104: Saturday day shift pull for VIL3 completed.
2021-07-30 00:41:41.147943: Saturday day shift pull for VMI1 completed.
2021-07-30 00:42:01.356693: Saturday day shift pull for VFL4 completed.
2021-07-30 00:42:14.164621: Saturday day shift pull request for VIL1 dropped, retry in place.
2021-07-30 00:42:25.930291: Saturday day shift pull for VIL1 completed.
2021-07-30 00:42:26.091674: Starting Volume pulls from last Friday day shift.
2021-07-30 00:42:38.454705: Friday day shift pull for VIL2 completed.
2021-07-30 00:42:46.148148: Friday day shift pull for VIL3 completed.
2021-07-30 00:42:50.086360: Friday day shift pull for VMI1 completed.
2021-07-30 00:42:53.964000: Friday day shift pull for VFL4 completed.
2021-07-30 00:42:57.601132: Friday day shift pull request for VIL1 dropped, retry in place.
2021-07-30 00:43

,index,VIL2,VIL3,VMI1,VFL4,VIL1
0,2021-07-24 06:30-20:00,10430.0,8022.0,9274.0,10729.0,0
1,2021-07-23 06:30-20:00,12450.0,9357.0,10047.0,11944.0,0
2,2021-07-22 06:30-20:00,11893.0,9005.0,10076.0,10903.0,0
3,2021-07-21 06:30-20:00,11948.0,9069.0,11984.0,10490.0,0
4,2021-07-20 06:30-20:00,12627.0,9343.0,10884.0,9922.0,0
5,2021-07-19 06:30-20:00,13155.0,11794.0,10934.0,9778.0,0
6,2021-07-18 06:30-20:00,11546.0,7270.0,10808.0,10033.0,0


In [535]:
def NightShiftVolume():
    if count == 1:     
        for timeG in range(7):
            lastSunday = getSaturday() - timedelta(days=timeG)
            strLastSaturday = lastSaturday.strftime('%Y-%m-%d')
            strLastSunday = lastSunday.strftime('%Y-%m-%d')

            
            weekDay = days[timeG]

            print(str(datetime.now())+": Starting Volume pulls from last", weekDay, "night shift.")
            date1 = strLastSunday
            year1, month1, day1 = map(str, strLastSunday.split('-'))
            time1 = "21:00"
            hour1, minute1 = map(str, time1.split(':'))

            date2 = lastSunday + timedelta(days=1)
            strDate2 = date2.strftime('%Y-%m-%d')
            year2, month2, day2 = map(str, strDate2.split('-'))
            
            time2 = "06:30"
            hour2, minute2 = map(str, time2.split(':'))
            
            dateTotal = str(date1)+ " "+str(time1)+"-" +str(time2)
            my_dict = {"DS":[], dateTotal:[]}

            for x in whid:
                fclm_url = f'http://fclm-portal.amazon.com/ppa/inspect/node/csv?nodeType=DS&warehouseId={x}&startDateDay=2021%2F04%2F30&startDateWeek=2021%2F06%2F06&startDateMonth=2021%2F04%2F01&maxIntradayDays=1&spanType=Intraday&startDateIntraday={year1}%2F{month1}%2F{day1}&startHourIntraday={hour1}&startMinuteIntraday={minute1}&endDateIntraday={year2}%2F{month2}%2F{day2}&endHourIntraday={hour2}&endMinuteIntraday={minute2}'
                req = requests.Session()
                resp = req.get(fclm_url, verify=False, allow_redirects=True, auth=HTTPKerberosAuth(mutual_authentication=OPTIONAL))
                data = io.StringIO(resp.text)
                df = pd.read_csv(data)
            #print(df.info)
                lfilter = df['Labor Process Name']=="Container Building" 
                df.where(lfilter, inplace=True)#1
                filter = df['Main Processes'] == 'In-Station Core'
                df.where(filter, inplace=True)
                DS_sum = df["Actual Units"].sum()

                if x == whid[-1] and today.isocalendar()[1] < 32:
                    if DS_sum == 0:
                        print(str(datetime.now())+":", weekDay, "night shift pull request for", x, "dropped, retry in place.")
                        req = requests.Session()
                        resp = req.get(fclm_url, verify=False, allow_redirects=True, auth=HTTPKerberosAuth(mutual_authentication=OPTIONAL))
                        data = io.StringIO(resp.text)
                        df = pd.read_csv(data)
                            #print(df.info)
                        lfilter = df['Labor Process Name']=="Container Building" 
                        df.where(lfilter, inplace=True)#2
                        filter = df['Main Processes'] == 'In-Station Core'
                        df.where(filter, inplace=True)
                        DS_sum = df["Actual Units"].sum()
                        #print(str(datetime.now())+":", weekDay, "day shift pull for", x, "completed.")

                else:
                    while DS_sum == 0:
                        print(str(datetime.now())+":", weekDay, "night shift pull request for", x, "dropped, retry in place.")
                        req = requests.Session()
                        resp = req.get(fclm_url, verify=False, allow_redirects=True, auth=HTTPKerberosAuth(mutual_authentication=OPTIONAL))
                        data = io.StringIO(resp.text)
                        df = pd.read_csv(data)
                        #print(df.info)
                        lfilter = df['Labor Process Name']=="Container Building" 
                        df.where(lfilter, inplace=True)#2
                        filter = df['Main Processes'] == 'In-Station Core'
                        df.where(filter, inplace=True)
                        DS_sum = df["Actual Units"].sum()
                print(str(datetime.now())+":", weekDay, "night shift pull for", x, "completed.")


                my_dict["DS"].append(x)
                my_dict[dateTotal].append(DS_sum)
                #timeG = timeG + 1

            Volume1 = pd.DataFrame.from_dict(my_dict, orient='index', columns = ['VIL2', 'VIL3', 'VMI1', 'VFL4', 'VIL1'])
            Volume1.drop(labels = 'DS', axis = 0, inplace=True)

            Volume1 = Volume1.reset_index()
            Volume1.to_csv(VolumePM[timeG])
            if timeG==6:
                Vol1 = pd.read_csv(VolumePM[0])
                Vol2 = pd.read_csv(VolumePM[1])
                Vol3 = pd.read_csv(VolumePM[2])
                Vol4 = pd.read_csv(VolumePM[3])
                Vol5 = pd.read_csv(VolumePM[4])
                Vol6 = pd.read_csv(VolumePM[5])
                Vol7 = pd.read_csv(VolumePM[6])
                df_nightShift = pd.concat([Vol1, Vol2, Vol3, Vol4, Vol5, Vol6, Vol7], ignore_index=True)
                df_nightShift = df_nightShift.drop(['Unnamed: 0'], axis=1)
                df_nightShift.to_csv("NightShift.csv")
                
                for x in VolumePM:
                    os.remove(x)
                    print(str(datetime.now())+":", x, "has been removed.")
                    
                

                return df_nightShift
v2 = NightShiftVolume()
v2
        

2021-07-30 00:47:33.621525: Starting Volume pulls from last Saturday night shift.
2021-07-30 00:47:41.682589: Saturday night shift pull for VIL2 completed.
2021-07-30 00:47:58.771976: Saturday night shift pull for VIL3 completed.
2021-07-30 00:48:15.848348: Saturday night shift pull for VMI1 completed.
2021-07-30 00:48:28.458912: Saturday night shift pull for VFL4 completed.
2021-07-30 00:48:36.072318: Saturday night shift pull request for VIL1 dropped, retry in place.
2021-07-30 00:48:49.722369: Saturday night shift pull for VIL1 completed.
2021-07-30 00:48:49.734797: Starting Volume pulls from last Friday night shift.
2021-07-30 00:48:53.578624: Friday night shift pull for VIL2 completed.
2021-07-30 00:49:08.181542: Friday night shift pull for VIL3 completed.
2021-07-30 00:49:09.084938: Friday night shift pull for VMI1 completed.
2021-07-30 00:49:12.442568: Friday night shift pull for VFL4 completed.
2021-07-30 00:49:22.180634: Friday night shift pull request for VIL1 dropped, retry 

,index,VIL2,VIL3,VMI1,VFL4,VIL1
0,2021-07-24 21:00-06:30,6170.0,3915.0,6241.0,5681.0,0
1,2021-07-23 21:00-06:30,5938.0,4728.0,6368.0,4896.0,0
2,2021-07-22 21:00-06:30,6119.0,3666.0,7102.0,6216.0,0
3,2021-07-21 21:00-06:30,6566.0,4610.0,6972.0,6008.0,0
4,2021-07-20 21:00-06:30,6558.0,4918.0,7822.0,6794.0,0
5,2021-07-19 21:00-06:30,6525.0,5452.0,8567.0,6391.0,0
6,2021-07-18 21:00-06:30,5711.0,5537.0,8380.0,5941.0,0


In [536]:
df_night = pd.read_csv("NightShift.csv")
df_day = pd.read_csv("DayShift.csv")

df_week = df_day.append(df_night, ignore_index=True)

df_week = df_week.drop(['Unnamed: 0'], axis=1)
df_week = df_week.set_index('index')

df_volume_sum = df_week.sum()
df_vol_wk = pd.DataFrame(df_volume_sum)
df_vol_wk.columns = ['Total Volume']

df_vol_wk.to_csv("RegionVolumeTotals.csv")

df_week = df_week.reset_index()
df_week[['date', 'time']] = df_week['index'].str.split(' ', 1, expand=True)
df_week[['start', 'end']] = df_week['time'].str.split('-', 1, expand=True)


df_week['date'] = pd.to_datetime(df_week['date'])
df_week['day'] = df_week['date'].dt.day_name()


df_week['shift'] = df_week['day'] +" "+ df_week['start']



df_week = df_week.drop(['index', 'VIL1', 'date', 'VIL3', 'VMI1', 'VFL4',
       'time', 'start', 'end', 'day'], axis=1)

df_week = df_week.pivot(columns='shift', values = ['VIL2'])#, index=['VIL2', 'VIL3', 'VMI1', 'VFL4'])#, values='VIL2')#, index='VIL2')
df_week = df_week.fillna(0)
df_week = df_week.astype(int)


df_week.to_csv('VIL2TotalWeek.csv')

df_VIL2 = pd.read_csv('VIL2TotalWeek.csv', header=None, index_col=None)
df_VIL2 = df_VIL2.drop([0], axis=1)

nu_VIL=df_VIL2.tail(15)
df_nuVIL = pd.DataFrame(nu_VIL)

new_header = df_nuVIL.iloc[0]
df_nuVIL = df_nuVIL[1:]
df_nuVIL.columns = new_header

df_nuVIL


df_nuVIL = df_nuVIL.astype(int)
df_nuVIL['FHD'] = df_nuVIL['Sunday 06:30']+df_nuVIL['Monday 06:30']+df_nuVIL['Tuesday 06:30']
df_nuVIL['FHN'] = df_nuVIL['Sunday 21:00']+df_nuVIL['Monday 21:00']+df_nuVIL['Tuesday 21:00']

df_nuVIL['BHD'] = df_nuVIL['Thursday 06:30']+df_nuVIL['Friday 06:30']+df_nuVIL['Saturday 06:30']
df_nuVIL['BHD'] = df_nuVIL['Thursday 21:00']+df_nuVIL['Friday 21:00']+df_nuVIL['Saturday 21:00']

df_nuVIL['WDD'] = df_nuVIL['Wednesday 06:30']
df_nuVIL['WDN'] = df_nuVIL['Wednesday 21:00']

df_nuVIL.loc['VIL2 Volume'] = df_nuVIL.sum(numeric_only=True, axis=0)
df_nuVIL = df_nuVIL.drop(['Friday 06:30', 'Friday 21:00', 'Monday 06:30', 'Monday 21:00',
       'Saturday 06:30', 'Saturday 21:00', 'Sunday 06:30', 'Sunday 21:00',
       'Thursday 06:30', 'Thursday 21:00', 'Tuesday 06:30', 'Tuesday 21:00',
       'Wednesday 06:30', 'Wednesday 21:00'], axis=1)
df_nuVIL

finalVol = df_nuVIL.tail(1)
df_final = pd.DataFrame(finalVol)

df_final.to_csv('VIL2 Volume.csv')

In [547]:
df_week

VIL2                                                        \
shift Friday 06:30 Friday 21:00 Monday 06:30 Monday 21:00 Saturday 06:30   
0                0            0            0            0          10430   
1            12450            0            0            0              0   
2                0            0            0            0              0   
3                0            0            0            0              0   
4                0            0            0            0              0   
5                0            0        13155            0              0   
6                0            0            0            0              0   
7                0            0            0            0              0   
8                0         5938            0            0              0   
9                0            0            0            0              0   
10               0            0            0            0              0   
11               0            0            0            0              0   
12               0            0            0         6525              0   
13               0            0            0            0              0   

                                                                              \
shift Saturday 21:00 Sunday 06:30 Sunday 21:00 Thursday 06:30 Thursday 21:00   
0                  0            0            0              0              0   
1                  0            0            0              0              0   
2                  0            0            0          11893              0   
3                  0            0            0              0              0   
4                  0            0            0              0              0   
5                  0            0            0              0              0   
6                  0        11546            0              0              0   
7               6170            0            0              0              0   
8                  0            0            0              0              0   
9                  0            0            0              0           6119   
10                 0            0            0              0              0   
11                 0            0            0              0              0   
12                 0            0            0              0              0   
13                 0            0         5711              0              0   

                                                                   
shift Tuesday 06:30 Tuesday 21:00 Wednesday 06:30 Wednesday 21:00  
0                 0             0               0               0  
1                 0             0               0               0  
2                 0             0               0               0  
3                 0             0           11948               0  
4             12627             0               0               0  
5                 0             0               0               0  
6                 0             0               0               0  
7                 0             0               0               0  
8                 0             0               0               0  
9                 0             0               0               0  
10                0             0               0            6566  
11                0          6558               0               0  
12                0             0               0               0  
13                0             0               0               0

In [548]:
df_nuVIL

1,FHD,FHN,BHD,WDD,WDN
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
5,0,0,0,11948,0
6,12627,0,0,0,0
7,13155,0,0,0,0
8,11546,0,0,0,0
9,0,0,6170,0,0
10,0,0,5938,0,0
11,0,0,6119,0,0


In [549]:
df_final

1,FHD,FHN,BHD,WDD,WDN
VIL2 Volume,37328,18794,18227,11948,6566
